In [ ]:
import pandas as pd

In [ ]:
# Load the datasets
df1 = pd.read_excel("/content/mizo_sentiment_dataset.xlsx")

In [ ]:
# Load the datasets
df2 = pd.read_excel("/content/Chungli_ao_Train.xlsx")

In [ ]:
df1.head()

,Text,Sentiment,__index_level_0__
0,Ruata of MPC pawh a tha khawp mai.,POSITIVE,1438
1,Ngaihtuahna tibuaitu leh khawtlang nun nena in...,POSITIVE,1256
2,"J Malsawma hi ka tan tlat,a thil tih toh hi hn...",POSITIVE,603
3,Performance tha tak tak leh narrative tha tak ...,POSITIVE,1466
4,Zopari hawiher hi ani huuuuu,NEGATIVE,3369


In [ ]:
# Concatenate the datasets
#df = df1
df = pd.concat([df1, df2])

In [ ]:
# Display information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11299 entries, 0 to 8578
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Text               11299 non-null  object 
 1   Sentiment          11299 non-null  object 
 2   __index_level_0__  2720 non-null   float64
dtypes: float64(1), object(2)
memory usage: 353.1+ KB


In [ ]:
# Converting the labels to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Sentiment'] = le.fit_transform(df.Sentiment.values)

In [ ]:
# Shuffle the dataframe
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
df

,Text,Sentiment,__index_level_0__
329,He film hian khawvel hmuhtir a tumna chu a hla...,0,3012.0
1663,mozü agitsü atema awareness Iba senden nung DT...,1,NaN
3724,kibong ka amai liasü. Nüburtem tenzüktsü nükla...,0,NaN
342,"tashidak agi shiranger shilem 85.01 (nisung 1,...",1,NaN
2260,Artist vocal nuances hian he hla hi complexity...,1,2518.0
...,...,...,...
615,ajak nung teka metsüktsü gari ajak nung passan...,1,NaN
2373,2021 (Agencies): Pungoben world chess champion...,0,NaN
2704,United States leniba nung alima tesem ajaklen ...,0,NaN
3889,"shiogo. German daily Bild-ia, Flick meletsüpur...",0,NaN


In [ ]:
# Count the number of true and fake labels
Positive_labels = (df['Sentiment'] == 1).sum()
Negative_labels = (df['Sentiment'] == 0).sum()
print(f"Number of   Postive labels: {Positive_labels}")
print(f"Number of Negative labels: {Negative_labels}")

Number of   Postive labels: 5875
Number of Negative labels: 5424


In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [ ]:
# Install required packages
!pip install sentencepiece
!pip install transformers
!pip install transformers accelerate
!pip install torch==1.11.00.676617

ERROR: Could not find a version that satisfies the requirement torch==1.11.00.676617 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch==1.11.00.676617


In [ ]:
# Load tokenizer and model
#from transformers import XLMRobertaTokenizer, XLMRobertaModel
#import torch
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
# # Here I loaded Bert tokenizer and MBERT model
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
# Define custom dataset class
class spanish_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Encode labels
train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

# Create dataset objects
train_dataset = spanish_Dataset(train_encodings, train_labels)
val_dataset = spanish_Dataset(val_encodings, val_labels)

In [ ]:
# additional classifocation layer on top of my model
import torch.nn as nn

class CustomClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(CustomClassifier, self).__init__()
        self.base_model = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the representation of [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch

In [ ]:
# Instantiate the custom model
#model_name = 'xlm-roberta-base'
#num_labels = 2  # Number of classes for classification
#custom_model = CustomClassifier(model_name, num_labels)

In [ ]:
#Here, I Instantiated the custom model
model_name = 'bert-base-multilingual-cased'
num_labels = 2  # classes in my data
custom_model = CustomClassifier(model_name, num_labels)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# Define metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc, 'F1': f1, 'Precision': precision, 'Recall': recall}

In [ ]:
!pip install wandb
import wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
# Here, I initialized the weights & biases
wandb.init(project='Roboust_nlp_mbert', name='Mizo+Chungli_ao_Second_run')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Define training arguments
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    learning_rate=0.0001,
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10,
    report_to='wandb',
)

# Initialize the Trainer
trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # compute metrics function
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.687300,0.704307,0.478761,0.478761,0.478761,0.478761
20,0.711600,0.677919,0.612389,0.612389,0.612389,0.612389
30,0.693200,0.688414,0.527876,0.527876,0.527876,0.527876
40,0.683400,0.676402,0.521681,0.521681,0.521681,0.521681
50,0.648300,0.608515,0.761947,0.761947,0.761947,0.761947
60,0.593800,0.502272,0.774779,0.774779,0.774779,0.774779
70,0.444700,0.448201,0.829204,0.829204,0.829204,0.829204
80,0.490400,0.365194,0.854425,0.854425,0.854425,0.854425
90,0.407700,0.350750,0.876106,0.876106,0.876106,0.876106
100,0.348600,0.406917,0.845575,0.845575,0.845575,0.845575


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.687300,0.704307,0.478761,0.478761,0.478761,0.478761
20,0.711600,0.677919,0.612389,0.612389,0.612389,0.612389
30,0.693200,0.688414,0.527876,0.527876,0.527876,0.527876
40,0.683400,0.676402,0.521681,0.521681,0.521681,0.521681
50,0.648300,0.608515,0.761947,0.761947,0.761947,0.761947
60,0.593800,0.502272,0.774779,0.774779,0.774779,0.774779
70,0.444700,0.448201,0.829204,0.829204,0.829204,0.829204
80,0.490400,0.365194,0.854425,0.854425,0.854425,0.854425
90,0.407700,0.350750,0.876106,0.876106,0.876106,0.876106
100,0.348600,0.406917,0.845575,0.845575,0.845575,0.845575


{'eval_loss': 0.18340568244457245, 'eval_Accuracy': 0.9504424778761061, 'eval_F1': 0.9504424778761061, 'eval_Precision': 0.9504424778761061, 'eval_Recall': 0.9504424778761061, 'eval_runtime': 33.2262, 'eval_samples_per_second': 68.019, 'eval_steps_per_second': 4.274, 'epoch': 2.0353982300884956}


In [16]:
test_df = pd.read_excel("/content/Chungli_Ao_test.xlsx")

In [17]:
len(test_df)

2000

In [18]:
test_df.head()

,Text,Sentiment
0,tajungtiba nung anir. ATMA staff dang o tajung...,POSITIVE
1,Iba osang atema onok dang agitsüpur ajak iba t...,POSITIVE
2,ni iba tasen ba ya meimer,POSITIVE
3,Aba tasayaba ya ajak dang tajungtiba,POSITIVE
4,Tajitettsütem Symptoms: (tang tashi nung angaz...,POSITIVE


In [19]:
test_df.tail()

,Text,Sentiment
1995,parnok kong aliar mejangja; süadoker asü tepuo...,NEGATIVE
1996,Magadh Range nunga DIG inyakogo.Taoba mapang C...,NEGATIVE
1997,(FLWs) aser arishi küm 45 ser temapur nem vacc...,NEGATIVE
1998,"Director, Rev Atha Neikha ajanga sayatsü. Iba ...",NEGATIVE
1999,melenshija lir.Democracy busema longkak aitba ...,NEGATIVE


In [20]:
test_texts = list(test_df['Text'])

In [21]:
test_df['Sentiment'] = le.fit_transform(test_df.Sentiment.values)

In [22]:
test_labels = test_df['Sentiment']

In [23]:
len(test_labels)

2000

In [24]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset =  CustomDataset(test_encodings, test_labels)

In [25]:
test_metrics = trainer.evaluate(test_dataset)

In [26]:
print(test_metrics)

{'eval_loss': 0.5198549628257751, 'eval_Accuracy': 0.8605, 'eval_F1': 0.8605, 'eval_Precision': 0.8605, 'eval_Recall': 0.8605, 'eval_runtime': 12.8596, 'eval_samples_per_second': 155.526, 'eval_steps_per_second': 9.72, 'epoch': 3.0}
